In [61]:
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import os
import scipy.io.wavfile as wav
import numpy as np
import pandas as pd
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Conv1D, MaxPooling1D
from keras.optimizers import RMSprop,SGD
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from keras import utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np


In [2]:
x=list()
y=list()
for subdir in os.listdir("genres/"):
    for f in os.listdir("genres/"+subdir+"/"):
        (rate,sig) = wav.read("genres/"+subdir+"/"+f)
        #print(rate)
        mfcc_feat = mfcc(sig,rate)
        fb_feat = logfbank(sig,rate)
        #reshaped=(np.reshape(mfcc_feat,(1,len(mfcc_feat)*13)))
        #l.append((subdir,f,reshaped))
        mfcc_df=pd.DataFrame(mfcc_feat)[250:2500]
        fb_df=pd.DataFrame(fb_feat)[250:2500]
        fields=pd.concat([mfcc_df,fb_df],axis=1)
        grouped=fields.groupby(fields.index//25)
        agg=grouped.mean()
        #agg=pd.concat([mfcc_df.max(),mfcc_df.min(),mfcc_df.std(),mfcc_df.mean(),mfcc_df.median()],axis=1)
        #reshaped_agg=agg.as_matrix().flatten()

        x.append(agg.as_matrix())
        y.append(subdir)
        #x.append(agg[:25].as_matrix())
        #x.append(agg[25:50].as_matrix())
        #x.append(agg[50:75].as_matrix())
        #x.append(agg[75:].as_matrix())
        #x.append(mfcc_df.as_matrix())
        #y.append(subdir)
        #y.append(subdir)
        #y.append(subdir)        
        #y.append(subdir)
            

In [23]:
x=np.asarray(x)
y=np.asarray(y)
unique_vals , y_cat=np.unique(y,return_inverse=True)
y_proc=utils.to_categorical(y_cat,num_classes=10)
#y=utils.to_categorical(y,num_classes=10)
print(x.shape)
print(y.shape)
print (unique_vals)
print (y[400],y_cat[400],y_proc[400])


(4000, 25, 39)
(4000,)
['blues' 'classical' 'country' 'disco' 'hiphop' 'jazz' 'metal' 'pop'
 'reggae' 'rock']
jazz 5 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]


In [34]:
x_train_cv,x_test_cv,y_train_cv,y_test_cv=model_selection.train_test_split(x,y_proc,test_size=500)

#print(y_train)
#np.unique(np.argmax(y_train,axis=1),return_counts=True)
print(np.unique(np.argmax(y_test_cv,axis=1),return_counts=True))
#np.argmax(y_train),return_counts=True))

kf = KFold(n_splits=10,shuffle=True,random_state=10)
kf.get_n_splits(x_train_cv)

print(kf)  

[[ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([39, 42, 36, 39, 40, 43, 43, 39, 43, 36]))
KFold(n_splits=10, random_state=10, shuffle=True)


In [68]:
def create_model(epochs):
    
    data_dim = 39
    timesteps = 90
    num_classes = 10

    # expected input data shape: (batch_size, timesteps, data_dim)
    model = Sequential()
    model.add(LSTM(500, return_sequences=True,
                   input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
    model.add(Dropout(.50))
    model.add(LSTM(250, return_sequences=True))  # returns a sequence of vectors of dimension 32
    #model.add(LSTM(250))  # returns a sequence of vectors of dimension 32
    model.add(Dropout(.50))
    model.add(LSTM(125))  # return a single vector of dimension 32
    #model.add(Dense(125, activation='relu'))
    #smodel.add(Dense(64,activation='relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    #defualt lr 0.001

    #decay params
    #epochs = 20
    learning_rate = 0.001
    #decay_rate = learning_rate / epochs

    opt=RMSprop(lr=learning_rate, rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [ ]:
def get_vote(models,x_test,y_test):
    #compile preds and have models vote. if tie - takes first example
    preds=list()
    for model in models:
        preds.append(model.predict_classes(x_test))
    print (preds)
    preds_df=pd.DataFrame(preds).T
    print (preds_df)
    model_vote=preds_df.mode(axis=1).fillna(preds_df[0])
    print (model_vote)

    results=pd.DataFrame({"true":np.argmax(y_test_cv,axis=1),"vote":model_vote[0]})
    print (results)
    total_accuracy=accuracy_score(results["true"],results["vote"])
    print (total_accuracy)
    for i in range(len(models)):
        single_model_accuracy=accuracy_score(results["true"],preds_df[i])
        print (single_model_accuracy)
    

In [ ]:
models=list()
i=0

In [67]:

for train_index, test_index in kf.split(x_train_cv):
   #print("TRAIN:", train_index, "TEST:", test_index)
   x_train, x_test = x_train_cv[train_index], x_train_cv[test_index]
   y_train, y_test = y_train_cv[train_index], y_train_cv[test_index] 
   print (x_train.shape)
   print (y_train.shape)
   epochs=40
   model=create_model(epochs)
   #model.fit(x_train, y_train,validation_data=(x_test,y_test),batch_size=100, epochs=epochs)
   model.fit(np.concatenate([x_train,x_test]), np.concatenate([y_train,y_test]),validation_data=(x_test_cv,y_test_cv),batch_size=50, epochs=epochs,verbose=1)
   models.append(model)
   if i>0:
    get_vote(models,x_test_cv,y_test_cv)
   #break
   i=i+1
   print (i)

(3240, 25, 39)
(3240, 10)
Train on 3240 samples, validate on 360 samples
Epoch 1/20
3240/3240 [==============================] - 22s - loss: 2.0059 - acc: 0.2957 - val_loss: 1.7920 - val_acc: 0.4250

KeyboardInterrupt: 

In [1]:
#compile preds and have models vote. if tie - takes first example
preds=list()
for model in models:
    preds.append(model.predict_classes(x_test_cv))
print (preds)
preds_df=pd.DataFrame(preds).T
print (preds_df)
model_vote=preds_df.mode(axis=1).fillna(preds_df[0])


NameError: name 'models' is not defined

In [66]:
results=pd.DataFrame({"true":np.argmax(y_test_cv,axis=1),"vote":model_vote[0]})
total_accuracy=accuracy_score(results["true"],results["vote"])
print (total_accuracy)
for i in range(len(models)):
    single_model_accuracy=accuracy_score(results["true"],preds_df[i])
    print (single_model_accuracy)

0.53


In [ ]:
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,y_proc,test_size=400)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

#x_train_int=((x_train+40)*100).astype(int)
x_train = x_train.reshape(3600,25,39)
x_test = x_test.reshape(400,25,39)

print(y_train)
np.unique(np.argmax(y_train,axis=1),return_counts=True)
print(np.unique(np.argmax(y_test,axis=1),return_counts=True))
#np.argmax(y_train),return_counts=True))